#### Document sources:

- [ardusub](https://www.ardusub.com/developers/pymavlink.html)
- [mavlink common message set](https://mavlink.io/en/messages/common.html)

### INSTALLATION <a name="INSTALLATION"></a>

BLUEROV2 runs "ARDUSUB" firmware on its external pixhawk.
The easiest way to communicate with the pixhawk is through the Python pymavlink library.

In this first step we will just go through the initial installation and setup of the pymavlink library.

### Ubuntu 20.04

In [ ]:
# Update list of available packages
sudo apt update
sudo apt -y upgrade

# Install some dependencies
sudo apt install -y python3-pip

# Install mavproxy module and everything else needed
pip3 install mavproxy

### Mac/Windows

With Python and pip installed, run pip install wheel mavproxy.

**Test installation**

You can test your installation with interactive shell

In [ ]:
python
import pymavlink
print(pymavlink.__doc__)

### Connection  methods <a name="connection"></a>

There are 3 types of udp connections for `mavlink_connection`:
- **udpout**: Outputs data to a specified address:port (client).
- **udpbcast**: Broadcasts and locks to the first client to respond (does not handle multiple clients properly)
  * Using the IP `192.168.1.255` all devices from `192.168.1.1` to `192.168.1.255` will receive the data.
- **udpin**: Binds to a specific port in address:port (server), it's necessary to receive data from clients (**udpout**) before starting to send any data.
- **udp**: Exists for legacy reasons, works as udpin

### Send commands <a name="send_commands"></a>

Below are multiple commands that you can execute to test your BLUEROV2

#### Import library

In [ ]:
# Import mavutil
from pymavlink import mavutil
import sys

# Create the connection
master = mavutil.mavlink_connection('udpin:0.0.0.0:14550')
# Wait a heartbeat before sending commands
master.wait_heartbeat()

#### Connect to the ROV from surface computer

In [ ]:
"""
Example of how to connect pymavlink to an autopilot via an UDP connection
"""

# Create the connection
#  If using a companion computer
#  the default connection is available
#  at ip 192.168.2.1 and the port 14550
# Note: The connection is done with 'udpin' and not 'udpout'.
#  You can check in http:192.168.2.2:2770/mavproxy that the communication made for 14550
#  uses a 'udpbcast' (client) and not 'udpin' (server).
#  If you want to use QGroundControl in parallel with your python script,
#  it's possible to add a new output port in http:192.168.2.2:2770/mavproxy as a new line.
#  E.g: --out udpbcast:192.168.2.255:yourport
master = mavutil.mavlink_connection('udpin:0.0.0.0:14550')

# Make sure the connection is valid
master.wait_heartbeat()

# Get some information !

start_time = time.time()

# Print info for 3 seconds
while time.time() - start_time < 3:
    try:
        print(master.recv_match().to_dict())
    except:
        pass
    time.sleep(0.1)

# Output:
# {'mavpackettype': 'AHRS2', 'roll': -0.11364290863275528, 'pitch': -0.02841472253203392, 'yaw': 2.0993032455444336, 'altitude': 0.0, 'lat': 0, 'lng': 0}
# {'mavpackettype': 'AHRS3', 'roll': 0.025831475853919983, 'pitch': 0.006112074479460716, 'yaw': 2.1514968872070312, 'altitude': 0.0, 'lat': 0, 'lng': 0, 'v1': 0.0, 'v2': 0.0, 'v3': 0.0, 'v4': 0.0}
# {'mavpackettype': 'VFR_HUD', 'airspeed': 0.0, 'groundspeed': 0.0, 'heading': 123, 'throttle': 0, 'alt': 3.129999876022339, 'climb': 3.2699999809265137}
# {'mavpackettype': 'AHRS', 'omegaIx': 0.0014122836291790009, 'omegaIy': -0.022567369043827057, 'omegaIz': 0.02394154854118824, 'accel_weight': 0.0, 'renorm_val': 0.0, 'error_rp': 0.08894175291061401, 'error_yaw': 0.0990816056728363}

#### ARM/DISARM the vehicle

**ARM**

In [ ]:
# master.arducopter_arm() or:
master.mav.command_long_send(
    master.target_system,
    master.target_component,
    mavutil.mavlink.MAV_CMD_COMPONENT_ARM_DISARM,
    0,
    1, 0, 0, 0, 0, 0, 0)

# wait until arming confirmed (can manually check with master.motors_armed())
print("Waiting for the vehicle to arm")
master.motors_armed_wait()
print('Armed!')

**DISARM**

In [ ]:
# master.arducopter_disarm() or:
master.mav.command_long_send(
    master.target_system,
    master.target_component,
    mavutil.mavlink.MAV_CMD_COMPONENT_ARM_DISARM,
    0,
    0, 0, 0, 0, 0, 0, 0)

# wait until disarming confirmed
master.motors_disarmed_wait()
print('Disarmed!')

#### Change flight mode

TODO: link to possible modes

GitHub Copilot: Sure, here are the possible modes for Ardusub:

- **ACRO**: Acrobatic mode
- **ALT_HOLD**: Altitude hold mode
- **AUTO**: Autonomous mode
- **BRAKE**: Brake mode
- **GUIDED**: Guided mode
- **LOITER**: Loiter mode
- **MANUAL**: Manual mode
- **POSHOLD**: Position hold mode
- **STABILIZE**: Stabilize mode

You can find more information about these modes in the [official ArduSub documentation](https://www.ardusub.com/operators-manual/rc-control.html#flight-modes).

In [ ]:
# Choose a mode
mode = 'STABILIZE'

# Check if mode is available
if mode not in master.mode_mapping():
    print('Unknown mode : {}'.format(mode))
    print('Try:', list(master.mode_mapping().keys()))
    sys.exit(1)

# Get mode ID
mode_id = master.mode_mapping()[mode]
# Set new mode
# master.mav.command_long_send(
#    master.target_system, master.target_component,
#    mavutil.mavlink.MAV_CMD_DO_SET_MODE, 0,
#    0, mode_id, 0, 0, 0, 0, 0) or:
# master.set_mode(mode_id) or:
master.mav.set_mode_send(
    master.target_system,
    mavutil.mavlink.MAV_MODE_FLAG_CUSTOM_MODE_ENABLED,
    mode_id)

while True:
    # Wait for ACK command
    # Would be good to add mechanism to avoid endlessly blocking
    # if the autopilot sends a NACK or never receives the message
    ack_msg = master.recv_match(type='COMMAND_ACK', blocking=True)
    ack_msg = ack_msg.to_dict()

    # Continue waiting if the acknowledged command is not `set_mode`
    if ack_msg['command'] != mavutil.mavlink.MAV_CMD_DO_SET_MODE:
        continue

    # Print the ACK result !
    print(mavutil.mavlink.enums['MAV_RESULT'][ack_msg['result']].description)
    break

#### Send RC commands (Joystick)

**Keep in mind that pixhawk RC controls need to be sent continuiously. A rate of 5hz or faster should work**

In [ ]:
"""
Example of how to use RC_CHANNEL_OVERRIDE messages to force input channels
in Ardupilot. These effectively replace the input channels (from joystick
or radio), NOT the output channels going to thrusters and servos.
"""

# Create a function to send RC values
# More information about Joystick channels
# here: https://www.ardusub.com/operators-manual/rc-input-and-output.html#rc-inputs
def set_rc_channel_pwm(channel_id, pwm=1500):
    """ Set RC channel pwm value
    Args:
        channel_id (TYPE): Channel ID
        pwm (int, optional): Channel pwm value 1100-1900
    """
    if channel_id < 1 or channel_id > 18:
        print("Channel does not exist.")
        return

    # Mavlink 2 supports up to 18 channels:
    # https://mavlink.io/en/messages/common.html#RC_CHANNELS_OVERRIDE
    rc_channel_values = [65535 for _ in range(18)]
    rc_channel_values[channel_id - 1] = pwm
    master.mav.rc_channels_override_send(
        master.target_system,                # target_system
        master.target_component,             # target_component
        *rc_channel_values)                  # RC channel list, in microseconds.


# Set some roll
set_rc_channel_pwm(2, 1600)

# Set some yaw
set_rc_channel_pwm(4, 1600)

# The camera pwm value sets the servo speed of a sweep from the current angle to
#  the min/max camera angle. It does not set the servo position.
# Set camera tilt to 45º (max) with full speed
set_rc_channel_pwm(8, 1900)

# Set channel 12 to 1500us
# This can be used to control a device connected to a servo output by setting the
# SERVO[N]_Function to RCIN12 (Where N is one of the PWM outputs)
set_rc_channel_pwm(12, 1500)

#### Send Manual Control

In [ ]:
"""
Example of how to send MANUAL_CONTROL messages to the autopilot using
pymavlink.
This message is able to fully replace the joystick inputs.
"""

# Send a positive x value, negative y, negative z,
# positive rotation and no button.
# https://mavlink.io/en/messages/common.html#MANUAL_CONTROL
# Warning: Because of some legacy workaround, z will work between [0-1000]
# where 0 is full reverse, 500 is no output and 1000 is full throttle.
# x,y and r will be between [-1000 and 1000].
master.mav.manual_control_send(
    master.target_system,
    500,
    -500,
    250,
    500,
    0)

# To active button 0 (first button), 3 (fourth button) and 7 (eighth button)
# It's possible to check and configure this buttons in the Joystick menu of QGC
buttons = 1 + 1 << 3 + 1 << 7
master.mav.manual_control_send(
    master.target_system,
    0,
    0,
    500, # 500 means neutral throttle
    0,
    buttons)

#### Set Target Depth / Attitude

**NOTE:** This command will only work if the ROV is in <u>depth hold mode</u>

In [ ]:
"""
Example of how to set target depth in depth hold mode with pymavlink
"""

import time
import math
# Import mavutil
from pymavlink import mavutil
# Imports for attitude
from pymavlink.quaternion import QuaternionBase

def set_target_depth(depth):
    """ Sets the target depth while in depth-hold mode.

    Uses https://mavlink.io/en/messages/common.html#SET_POSITION_TARGET_GLOBAL_INT

    'depth' is technically an altitude, so set as negative meters below the surface
        -> set_target_depth(-1.5) # sets target to 1.5m below the water surface.

    """
    master.mav.set_position_target_global_int_send(
        int(1e3 * (time.time() - boot_time)), # ms since boot
        master.target_system, master.target_component,
        coordinate_frame=mavutil.mavlink.MAV_FRAME_GLOBAL_INT,
        type_mask=( # ignore everything except z position
            mavutil.mavlink.POSITION_TARGET_TYPEMASK_X_IGNORE |
            mavutil.mavlink.POSITION_TARGET_TYPEMASK_Y_IGNORE |
            # DON'T mavutil.mavlink.POSITION_TARGET_TYPEMASK_Z_IGNORE |
            mavutil.mavlink.POSITION_TARGET_TYPEMASK_VX_IGNORE |
            mavutil.mavlink.POSITION_TARGET_TYPEMASK_VY_IGNORE |
            mavutil.mavlink.POSITION_TARGET_TYPEMASK_VZ_IGNORE |
            mavutil.mavlink.POSITION_TARGET_TYPEMASK_AX_IGNORE |
            mavutil.mavlink.POSITION_TARGET_TYPEMASK_AY_IGNORE |
            mavutil.mavlink.POSITION_TARGET_TYPEMASK_AZ_IGNORE |
            # DON'T mavutil.mavlink.POSITION_TARGET_TYPEMASK_FORCE_SET |
            mavutil.mavlink.POSITION_TARGET_TYPEMASK_YAW_IGNORE |
            mavutil.mavlink.POSITION_TARGET_TYPEMASK_YAW_RATE_IGNORE
        ), lat_int=0, lon_int=0, alt=depth, # (x, y WGS84 frame pos - not used), z [m]
        vx=0, vy=0, vz=0, # velocities in NED frame [m/s] (not used)
        afx=0, afy=0, afz=0, yaw=0, yaw_rate=0
        # accelerations in NED frame [N], yaw, yaw_rate
        #  (all not supported yet, ignored in GCS Mavlink)
    )

def set_target_attitude(roll, pitch, yaw):
    """ Sets the target attitude while in depth-hold mode.

    'roll', 'pitch', and 'yaw' are angles in degrees.

    """
    master.mav.set_attitude_target_send(
        int(1e3 * (time.time() - boot_time)), # ms since boot
        master.target_system, master.target_component,
        # allow throttle to be controlled by depth_hold mode
        mavutil.mavlink.ATTITUDE_TARGET_TYPEMASK_THROTTLE_IGNORE,
        # -> attitude quaternion (w, x, y, z | zero-rotation is 1, 0, 0, 0)
        QuaternionBase([math.radians(angle) for angle in (roll, pitch, yaw)]),
        0, 0, 0, 0 # roll rate, pitch rate, yaw rate, thrust
    )

# Create the connection
master = mavutil.mavlink_connection('udpin:0.0.0.0:14550')
boot_time = time.time()
# Wait a heartbeat before sending commands
master.wait_heartbeat()

# arm ArduSub autopilot and wait until confirmed
master.arducopter_arm()
master.motors_armed_wait()

# set the desired operating mode
DEPTH_HOLD = 'ALT_HOLD'
DEPTH_HOLD_MODE = master.mode_mapping()[DEPTH_HOLD]
while not master.wait_heartbeat().custom_mode == DEPTH_HOLD_MODE:
    master.set_mode(DEPTH_HOLD)

# set a depth target
set_target_depth(-0.5)

# go for a spin
# (set target yaw from 0 to 500 degrees in steps of 10, one update per second)
roll_angle = pitch_angle = 0
for yaw_angle in range(0, 500, 10):
    set_target_attitude(roll_angle, pitch_angle, yaw_angle)
    time.sleep(1) # wait for a second

# spin the other way with 3x larger steps
for yaw_angle in range(500, 0, -30):
    set_target_attitude(roll_angle, pitch_angle, yaw_angle)
    time.sleep(1)

# clean up (disarm) at the end
master.arducopter_disarm()
master.motors_disarmed_wait()

### RECEIVE DATA <a name="receive_data"></a>

To receive data we need to continuously read the mavlink messages from the ROV.
For each message we can acces the type using `msg.get_type()` and the data using `msg.to_dict()`.
Once we have that information, we can process the messages accordingly



#### The potential message types for ardusub are below. 
**They are formated as follows:**
- `Message type`: `Message description`
    - `field name`: `type`, `units`, `description` 
-------------------
- `VFR_HUD`: VFR HUD
    - `airspeed`: float, m/s, Current airspeed in m/s
    - `groundspeed`: float, m/s, Current ground speed in m/s
    - `heading`: int16_t, deg, Current heading in degrees, in compass units (0..360, 0=north)
    - `throttle`: float, %, Current throttle setting in % of max throttle
    - `alt`: float, m, Current altitude (MSL), in meters
    - `climb`: float, m/s, Current climb rate in meters/second

- `ATTITUDE`: Attitude of the ROV in quaternion format
    - `time_boot_ms`: uint32_t, ms, Timestamp (time since system boot).
    - `roll`: float, rad, Roll angle (-pi..+pi)
    - `pitch`: float, rad, Pitch angle (-pi..+pi)
    - `yaw`: float, rad, Yaw angle (-pi..+pi)
    - `rollspeed`: float, rad/s, Roll angular speed
    - `pitchspeed`: float, rad/s, Pitch angular speed
    - `yawspeed`: float, rad/s, Yaw angular speed

- `ATTITUDE_TARGET`: Attitude target of the ROV in quaternion format
    - `time_boot_ms`: uint32_t, ms, Timestamp (time since system boot).
    - `type_mask`: uint8_t, NA, Bitmap to indicate which dimensions should be ignored by the vehicle.
    - `q`: float[4], NA, Attitude quaternion (w, x, y, z order, zero-rotation is 1, 0, 0, 0)
    - `body_roll_rate`: float, rad/s, Body roll rate
    - `body_pitch_rate`: float, rad/s, Body pitch rate
    - `body_yaw_rate`: float, rad/s, Body yaw rate
    - `thrust`: float, NA, Collective thrust, normalized to 0 .. 1 (-1 .. 1 for vehicles capable of reverse trust)

- `BATTERY_STATUS`: Battery status of the ROV
    - `id`: uint8_t, NA, Battery ID
    - `battery_function`: uint8_t, NA, Function of the battery
    - `type`: uint8_t, NA, Type (chemistry) of the battery
    - `temperature`: int16_t, cdegC, Temperature of the battery in centi-degrees celsius. INT16_MAX for unknown temperature.
    - `voltages`: uint16_t[10], mV, Battery voltage of cells. Cells above the valid cell count for this battery should have the UINT16_MAX value.
    - `current_battery`: int16_t, cA, Battery current, -1: autopilot does not measure the current

- `EKF_STATUS_REPORT`: EKF status report
    - `flags`: uint16_t, NA, Flags
    - `velocity_variance`: float, m/s, Velocity variance
    - `pos_horiz_variance`: float, m, Horizontal Position variance
    - `pos_vert_variance`: float, m, Vertical Position variance
    - `compass_variance`: float, NA, Compass variance
    - `terrain_alt_variance`: float, m, Terrain Altitude variance
    - `air_speed_variance`: float, m/s, Air Speed variance

- `GLOBAL_POSITION_INT`: Global position of the ROV
    - `time_boot_ms`: uint32_t, ms, Timestamp (time since system boot).
    - `lat`: int32_t, degE7, Latitude, expressed as degrees * 1E7
    - `lon`: int32_t, degE7, Longitude, expressed as degrees * 1E7
    - `alt`: int32_t, mm, Altitude in meters, expressed as * 1000 (millimeters)
    - `relative_alt`: int32_t, mm, Altitude above ground in meters, expressed as * 1000 (millimeters)
    - `vx`: int16_t, cm/s, Ground X Speed (Latitude), expressed as m/s * 100
    - `vy`: int16_t, cm/s, Ground Y Speed (Longitude), expressed as m/s * 100
    - `vz`: int16_t, cm/s, Ground Z Speed (Altitude), expressed as m/s * 100
    - `hdg`: uint16_t, cdeg, Compass heading in degrees * 100, 0.0..359.99 degrees. If unknown, set to: UINT16_MAX

- `HEARTBEAT`: Heartbeat of the ROV
    - `type`: uint8_t, NA, Type of the MAV (quadrotor, helicopter, etc., up to 15 types, defined in MAV_TYPE ENUM)
    - `autopilot`: uint8_t, NA, Autopilot type / class. defined in MAV_AUTOPILOT ENUM
    - `base_mode`: uint8_t, NA, System mode bitfield, see MAV_MODE_FLAGS ENUM in mavlink/include/mavlink_types.h
    - `custom_mode`: uint32_t, NA, A bitfield for use for autopilot-specific flags.
    - `system_status`: uint8_t, NA, System status flag, see MAV_STATE ENUM
    - `mavlink_version`: uint8_t, NA, MAVLink version, not writable by user, gets added by protocol because of magic data type: uint8_t_mavlink

- `LOCAL_POSITION_NED`: Local position of the ROV
    - `time_boot_ms`: uint32_t, ms, Timestamp (time since system boot).
    - `x`: float, m, X Position
    - `y`: float, m, Y Position
    - `z`: float, m, Z Position
    - `vx`: float, m/s, X Speed
    - `vy`: float, m/s, Y Speed
    - `vz`: float, m/s, Z Speed

- `MANUAL_CONTROL`: Manual control of the ROV
    - `time_boot_ms`: uint32_t, ms, Timestamp (time since system boot).
    - `x`: int16_t, NA, X-axis, normalized to the range [-1000, 1000]. A value of INT16_MAX indicates that this axis is invalid.
    - `y`: int16_t, NA, Y-axis, normalized to the range [-1000, 1000]. A value of INT16_MAX indicates that this axis is invalid.
    - `z`: int16_t, NA, Z-axis, normalized to the range [-1000, 1000]. A value of INT16_MAX indicates that this axis is invalid.
    - `r`: int16_t, NA, R-axis, normalized to the range [-1000, 1000]. A value of INT16_MAX indicates that this axis is invalid.
    - `buttons`: uint16_t, NA, A bitfield corresponding to the joystick buttons' current state, 1 for pressed, 0 for released. The lowest bit corresponds to Button 1.

- `MANUAL_SETPOINT`: Manual setpoint of the ROV
    - `time_boot_ms`: uint32_t, ms, Timestamp (time since system boot).
    - `roll`: float, rad, Desired roll rate
    - `pitch`: float, rad, Desired pitch rate
    - `yaw`: float, rad, Desired yaw rate
    - `thrust`: float, NA, Collective thrust, normalized to 0 .. 1 (-1 .. 1 for vehicles capable of reverse trust)

- `NAV_CONTROLLER_OUTPUT`: Navigation controller output
    - `nav_roll`: float, rad, Current desired roll in radians
    - `nav_pitch`: float, rad, Current desired pitch in radians
    - `nav_bearing`: int16_t, deg, Current desired heading in degrees
    - `target_bearing`: int16_t, deg, Bearing to current waypoint/target in degrees
    - `wp_dist`: uint16_t, m, Distance to active waypoint in meters
    - `alt_error`: float, m, Current altitude error in meters
    - `aspd_error`: float, m/s, Current airspeed error in meters/second
    - `xtrack_error`: float, m, Current crosstrack error on x-y plane in meters

- `SCALED_IMU2`: Scaled IMU2
    - `time_boot_ms`: uint32_t, ms, Timestamp (time since system boot).
    - `xacc`: int16_t, mG, X acceleration (mg)
    - `yacc`: int16_t, mG, Y acceleration (mg)
    - `zacc`: int16_t, mG, Z acceleration (mg)
    - `xgyro`: int16_t, mrad/s, Angular speed around X axis (millirad /sec)
    - `ygyro`: int16_t, mrad/s, Angular speed around Y axis (millirad /sec)
    - `zgyro`: int16_t, mrad/s, Angular speed around Z axis (millirad /sec)
    - `xmag`: int16_t, mgauss, X Magnetic field (milli Gauss)
    - `ymag`: int16_t, mgauss, Y Magnetic field (milli Gauss)
    - `zmag`: int16_t, mgauss, Z Magnetic field (milli Gauss)

- `SCALED_PRESSURE`: Scaled pressure
    - `time_boot_ms`: uint32_t, ms, Timestamp (time since system boot).
    - `press_abs`: float, hPa, Absolute pressure (hectopascal)
    - `press_diff`: float, hPa, Differential pressure 1 (hectopascal)
    - `temperature`: int16_t, cdegC, Temperature measurement (0.01 degrees celsius)

- `SCALED_PRESSURE2`: Scaled pressure2
    - `time_boot_ms`: uint32_t, ms, Timestamp (time since system boot).
    - `press_abs`: float, hPa, Absolute pressure (hectopascal)
    - `press_diff`: float, hPa, Differential pressure 2 (hectopascal)
    - `temperature`: int16_t, cdegC, Temperature measurement (0.01 degrees celsius)

- `SYS_STATUS`: System status
    - `onboard_control_sensors_present`: uint32_t, NA, Bitmap showing which onboard controllers and sensors are present. Value of 0: not present. Value of 1: present.
    - `onboard_control_sensors_enabled`: uint32_t, NA, Bitmap showing which onboard controllers and sensors are enabled: Value of 0: not enabled. Value of 1: enabled.
    - `onboard_control_sensors_health`: uint32_t, NA, Bitmap showing which onboard controllers and sensors have an error (or are operational). Value of 0: error. Value of 1: healthy.
    - `load`: uint16_t, %, Maximum usage in percent of the mainloop time, (0%: 0, 100%: 10000) if available on the platform
    - `voltage_battery`: uint16_t, mV, Battery voltage, in millivolts (1 = 1 millivolt)
    - `current_battery`: int16_t, cA, Battery current, in centiamperes (1 = 10 milliampere), -1: autopilot does not measure the current
    - `battery_remaining`: int8_t, %, Remaining battery energy: (0%: 0, 100%: 100), -1: autopilot estimate the remaining battery

- `RC_CHANNELS`: RC channels
    - `time_boot_ms`: uint32_t, ms, Timestamp (time since system boot).
    - `chancount`: uint8_t, NA, Total number of RC channels being received. This can be larger than 18, indicating that more channels are available but not given in this message.
    - `chan1_raw`: uint16_t, NA, RC channel 1 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan2_raw`: uint16_t, NA, RC channel 2 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan3_raw`: uint16_t, NA, RC channel 3 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan4_raw`: uint16_t, NA, RC channel 4 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan5_raw`: uint16_t, NA, RC channel 5 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan6_raw`: uint16_t, NA, RC channel 6 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan7_raw`: uint16_t, NA, RC channel 7 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan8_raw`: uint16_t, NA, RC channel 8 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan9_raw`: uint16_t, NA, RC channel 9 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan10_raw`: uint16_t, NA, RC channel 10 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan11_raw`: uint16_t, NA, RC channel 11 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan12_raw`: uint16_t, NA, RC channel 12 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan13_raw`: uint16_t, NA, RC channel 13 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan14_raw`: uint16_t, NA, RC channel 14 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan15_raw`: uint16_t, NA, RC channel 15 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan16_raw`: uint16_t, NA, RC channel 16 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan17_raw`: uint16_t, NA, RC channel 17 value, in microseconds. A value of UINT16_MAX implies the channel is unused.
    - `chan18_raw`: uint16_t, NA, RC channel 18 value, in microseconds. A value of UINT16_MAX implies the channel is unused.

#### Receive data by message type example

In [ ]:
"""
Example of how to filter for specific mavlink messages coming from the
autopilot using pymavlink.

Can also filter within recv_match command - see "Read all parameters" example
"""

while True:
    msg = master.recv_match()
    if not msg:
        continue
    if msg.get_type() == 'HEARTBEAT':
        print("\n\n*****Got message: %s*****" % msg.get_type())
        print("Message: %s" % msg)
        print("\nAs dictionary: %s" % msg.to_dict())
        # Armed = MAV_STATE_STANDBY (4), Disarmed = MAV_STATE_ACTIVE (3)
        print("\nSystem status: %s" % msg.system_status)
    elif msg.get_type() == 'VFR_HUD':
        print("\n\n*****Got message: %s*****" % msg.get_type())
        print("Message: %s" % msg)
        print("\nAs dictionary: %s" % msg.to_dict())
        print("\nAltitude: %s" % msg.alt)
        print("\nGroundspeed: %s" % msg.groundspeed)
        print("\nHeading: %s" % msg.heading)
        print("\nThrottle: %s" % msg.throttle)
    elif msg.get_type() == 'ATTITUDE':
        print("\n\n*****Got message: %s*****" % msg.get_type())
        print("Message: %s" % msg)
        print("\nAs dictionary: %s" % msg.to_dict())
        print("\nRoll: %s" % msg.roll)
        print("\nPitch: %s" % msg.pitch)
        print("\nYaw: %s" % msg.yaw)